# Imports

In [1]:
!pip install simpletransformers -q

     |████████████████████████████████| 204kB 26.7MB/s 
     |████████████████████████████████| 1.1MB 47.0MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 7.4MB 46.3MB/s 
     |████████████████████████████████| 317kB 57.3MB/s 
     |████████████████████████████████| 1.8MB 16.6MB/s 
     |████████████████████████████████| 2.9MB 45.3MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 1.4MB 49.8MB/s 
     |████████████████████████████████| 102kB 13.7MB/s 
     |████████████████████████████████| 112kB 55.2MB/s 
     |████████████████████████████████| 163kB 46.5MB/s 
     |████████████████████████████████| 4.5MB 45.7MB/s 
     |████████████████████████████████| 133kB 59.2MB/s 
     |████████████████████████████████| 102kB 14.4MB/s 
     |████████████████████████████████| 890kB 47.1MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 122kB 57.9MB

In [2]:
import pandas as pd
import numpy as np
import torch 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import wandb

# Load data

In [3]:
from google.colab import drive
import glob\

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# CHANGE TO YOUR PATH
colab_resources_path = "/content/gdrive/MyDrive/projectml"

In [5]:
data_files = glob.glob(colab_resources_path + "/*.csv")
data_files += glob.glob(colab_resources_path + "/*.py")
for data_file in data_files:
  print('Copying file {} to colab root.'.format(data_file))
  !cp "$data_file" .

Copying file /content/gdrive/MyDrive/projectml/random.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/nam.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/am.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/am_additional.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/missclassified_roberta_mcc.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/miss_5models.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/missclassified_lr_robertas_ens_mcc.csv to colab root.
Copying file /content/gdrive/MyDrive/projectml/data_preprocess.py to colab root.


Get four different datasets from train data:


*   `train_data_title` - dataframe with article titles
*   `train_data_body` - dataframe will article bodies
*   `train_data_titleplus` - dataframe with article title + first sentence
*   `train_data_all` - dataframe with article title + body



In [6]:
from data_preprocess import getTrainData
train_data_title = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=False, n_sentences=0) #for roberta on titles only
train_data_body = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=True, n_sentences=-1) #for roberta on articles only
train_data_titleplus = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=False, n_sentences=1) #for roberta on title+ 1 paraghaph only
train_data_all = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=False, n_sentences=-1) #for roberta on all

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



# Cross-validation for RoBERTa trained on different inputs

Specify classification model hyperparameters 

In [7]:
model_args = ClassificationArgs(sliding_window=True)
model_args.num_train_epochs=4
model_args.save_best_model= True
model_args.tie_value = 1
model_args.batch_size = 16
model_args.learning_rate = 2e-5
model_args.overwrite_output_dir = True
model_args.max_seq_length = 512
model_args.no_cache=True
model_args.max_grad_norm = 1
model_args.use_multiprocessing = True
model_args.manual_seed = 4
model_args.reprocess_input_data = True
model_args.evaluate_during_training = True
model_args.labels_list = [0, 1]

Function for memory cleanup, useful if several models are trained in one notebook.

In [8]:
import gc
def memory_cleanup():
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            del obj
    gc.collect()
    torch.cuda.empty_cache()
    !rm -r outputs

6-fold cross-validation, four models are trained at the same time. BERT-base-uncased is trained on:

*   full data
*   title only
*   text only
*   title + first sentence

In [10]:
n=6
seed=42
kf = KFold(n_splits=n, random_state=seed, shuffle=True)
acc, prec, rec, f1, mccs, wrongs = [[] for i in range(4)], [[] for i in range(4)],[[] for i in range(4)],[[] for i in range(4)],[[] for i in range(4)],[[] for i in range(4)]
miss_rb = [np.array([]) for i in range(4)]
predictions_rb=[[] for i in range(4)]
for train_index, val_index in kf.split(train_data_title): 
    memory_cleanup()

    train_df_t = train_data_title.iloc[train_index]
    val_df_t = train_data_title.iloc[val_index]
    train_df_b = train_data_body.iloc[train_index]
    val_df_b = train_data_body.iloc[val_index]
    train_df_tp = train_data_titleplus.iloc[train_index]
    val_df_tp = train_data_titleplus.iloc[val_index]
    train_df_a = train_data_all.iloc[train_index]
    val_df_a = train_data_all.iloc[val_index]

    inputs = [[train_df_t,val_df_t],[train_df_b,val_df_b],[train_df_tp,val_df_tp],[train_df_a,val_df_a]]
    for num, data in enumerate(inputs):

        model = ClassificationModel('bert', 'bert-base-uncased', args=model_args)
        model.train_model(data[0], eval_df=data[1], acc=matthews_corrcoef)
        result, model_outputs, wrong_predictions = model.eval_model(data[1], acc=matthews_corrcoef) 

        predictions_rb[num] = np.array([np.rint(np.mean(np.argmax(j, axis=1))) for j in model_outputs]).astype(int)

        miss_rb[num] = np.concatenate((miss_rb[num], data[1].iloc[np.where(data[1].label != predictions_rb[num])].index)).astype(int)

        acc[num].append(accuracy_score(data[1].label, predictions_rb[num]))
        prec[num].append(precision_score(data[1].label, predictions_rb[num]))
        rec[num].append(recall_score(data[1].label, predictions_rb[num]))
        f1[num].append(f1_score(data[1].label, predictions_rb[num]))
        mccs[num].append(matthews_corrcoef(data[1].label, predictions_rb[num]))
        memory_cleanup()

/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:126: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


rm: cannot remove 'outputs': No such file or directory


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:387: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


...

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:126: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from t

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:126: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


In [11]:
for num, i in enumerate(['title', 'body', 'title_plus', 'all']):
  print(i)
  print('Accuracy score: ', np.mean(acc[num]))
  print('Precision score: ', np.mean(prec[num]))
  print('Recall score: ', np.mean(rec[num]))
  print('F1 score: ', np.mean(f1[num]))
  print('mcc score: ', np.mean(mccs[num]))

title
Accuracy score:  0.8313614224239845
Precision score:  0.8206605297849855
Recall score:  0.8519598085876572
F1 score:  0.8357026973206518
mcc score:  0.6630909011391862
body
Accuracy score:  0.9354281931243204
Precision score:  0.9408558895499194
Recall score:  0.9298190644794534
F1 score:  0.9351946042388256
mcc score:  0.8708875204854802
title_plus
Accuracy score:  0.9040975079207452
Precision score:  0.9156192573635015
Recall score:  0.894057537453834
F1 score:  0.9041498359736665
mcc score:  0.8094227942777819
all
Accuracy score:  0.9385610252045208
Precision score:  0.9339007663839792
Recall score:  0.9453992814210862
F1 score:  0.9394195874420889
mcc score:  0.8774266574377578


In [12]:
for num, i in enumerate(['title', 'body', 'title_plus', 'all']):
  print(i)
  print(acc[num])
  print(prec[num])
  print(rec[num])
  print(f1[num])
  print(mccs[num])

title
[0.8195488721804511, 0.8458646616541353, 0.8458646616541353, 0.8233082706766918, 0.8007518796992481, 0.8528301886792453]
[0.8111888111888111, 0.8294573643410853, 0.8082191780821918, 0.8074074074074075, 0.8040540540540541, 0.8636363636363636]
[0.8467153284671532, 0.8492063492063492, 0.9007633587786259, 0.8384615384615385, 0.8321678321678322, 0.8444444444444444]
[0.8285714285714285, 0.8392156862745098, 0.851985559566787, 0.8226415094339623, 0.8178694158075601, 0.853932584269663]
[0.6389332404863596, 0.691417060604744, 0.6966132468860277, 0.6471935594162299, 0.5985260433624889, 0.7058622560792679]
body
[0.9285714285714286, 0.9398496240601504, 0.9210526315789473, 0.9323308270676691, 0.9473684210526315, 0.9433962264150944]
[0.9402985074626866, 0.9583333333333334, 0.9166666666666666, 0.9242424242424242, 0.951048951048951, 0.9545454545454546]
[0.9197080291970803, 0.9126984126984127, 0.9236641221374046, 0.9384615384615385, 0.951048951048951, 0.9333333333333333]
[0.929889298892989, 0.9349

In [13]:
import pickle

with open('miss_b.pkl', 'wb') as file:
  pickle.dump(miss_rb, file)

In [ ]:
from data_preprocess import getFullRowByIndex

miss_indexes = np.unique(np.concatenate(miss_rb,axis=1)).astype(int)

miss_data = getFullRowByIndex(miss_indexes)

for num, i in enumerate(['title', 'body', 'title_plus', 'all']):
  miss_data['miss_rb_'+i] = np.zeros(len(miss_data.index)).astype(int)

  miss_data.loc[miss_data.loc[miss_rb[num]].index, 'miss_rb_'+i] = 1

miss_data = miss_data.drop(columns=['assessor.1', 'article.1'])

try:
  miss_data.to_csv("/content/drive/My Drive/projectml/missclassified_bert_mcc.csv")
except:
  miss_data.to_csv("/content/gdrive/MyDrive/projectml/missclassified_bert_mcc.csv")

# Conclusion

BERT results are lower than results of RoBERTa for all four inputs. It is also interesting that performance of BERT on title is worse than performance of RoBERTa. As we expect overlap between articles misclassified by BERT and by RoBERTa, we chose RoBERTa for ensemble.